# dependencies
* install anaconda is recommended

```
cassandra-driver          3.11.0                   py35_1    conda-forge
pandas                    0.19.1              np111py35_0
scikit-learn              0.18.1              np111py35_0
scipy                     0.18.1              np111py35_0
matplotlib                2.0.0               np111py35_0
```

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import matplotlib
import time
import numpy as np
%matplotlib inline

import pandas as pd
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100

# import training as training

In [2]:
#Model File Directory
WORK_DIR = '/var/spark/ml_files/'

#Label Keys
LABEL = "success"

MODEL_TYPE = "ML-BR"

# start_date = '2018-01-01'
# end_date = '2018-03-30'
# site_ids = ['adbehap']
# site_ids = ['avast', 'kasperus', 'adbehap', 'adbehbr', 'adbehkr', 'mcafeeus', 'mfeap', 'mfeeu']
# desc = str('%s_%s_%s' % (start_date, end_date, "_".join(site_ids)))

In [3]:
# from cassandra
from cassandra.cluster import Cluster
cassandra_endpoint = '10.62.1.118'
cluster = Cluster([cassandra_endpoint])
# session = cluster.connect('subs')


In [4]:
import datetime
from datetime import timedelta
import time

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import preprocessing
import calendar, datetime
import time
from sklearn.preprocessing import LabelEncoder

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))




In [31]:
import itertools

subs_session = cluster.connect('subs')
df_cs_original = None
def get_site_date_cartesian_products(site_ids, dates):
    return itertools.product(*[site_ids, dates])

def query_all_sites(dates):
    """ Queries cpg raw data based on the given start, end date and all site_ids """
    query = "SELECT * FROM cpg_transaction where received_date = '%s'"
    _result = pd.DataFrame()
    for date in dates:
        query_by_date = str(query % date.replace('-',''))
        print(query_by_date)
        _result = pd.concat([_result, pd.DataFrame(list(subs_session.execute(query_by_date)))] )
        
    return _result

def query_by_site(dates, site_ids):
    """ Queries cpg raw data based on the given start, end date and list of site_ids """
    query = "SELECT * FROM cpg_transaction where received_date = '%s' and site_id = '%s'"    
    _result = pd.DataFrame()
    for site_id, date in get_site_date_cartesian_products(site_ids, dates):
        query_by_date = str(query % (date.replace('-',''), site_id))
        print(query_by_date)
        _result = pd.concat([_result, pd.DataFrame(list(subs_session.execute(query_by_date)))] )
    
    return _result

def query_raw_data(start, end, site_ids):
    """ Queries cpg raw data based on the given start, end date and list of site_ids """
    
#     _result = pd.DataFrame()
    dates = daterange(start, end)
    
    if not site_ids:
        return query_all_sites(dates)
        
    else:
        return query_by_site(dates, site_ids)

""" Load the raw data into df_cs_original """
df_cs_original = query_raw_data(start_date, end_date, site_ids)
df_cs_original.shape

SELECT * FROM cpg_transaction where received_date = '20180101' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180102' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180103' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180104' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180105' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180106' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180107' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180108' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180109' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180110' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where received_date = '20180111' and site_id = 'adbehap'
SELECT * FROM cpg_transaction where receive

(772710, 47)

In [6]:
csv_name = str('df_cs_original_%s_%s_%s.csv' % (start_date, end_date, "_".join(site_ids)))
csv_file_name = WORK_DIR + csv_name
df_cs_original.to_csv(csv_file_name)

csv_file_name

(3398837, 47)

In [48]:
df_cs_original['site_id'].value_counts()

avast       3368772
adbehap     1543648
kasperus    1118450
adbehkr      649196
adbehbr      605376
mfeeu        112575
mfeap         17979
mcafeeus        116
Name: site_id, dtype: int64

In [17]:
#Combine two df from csv
df_cs_original = None
file_1 = 'subs_retry_2018-01-01_2018-03-31_avast_kasperus_adbehap_adbehbr_adbehkr_mcafeeus_mfeap_mfeeu.csv'
retry_success_1 = pd.read_csv(WORK_DIR + file_1)

file_2 = 'subs_retry_2018-04-01_2018-06-30_avast_kasperus_adbehap_adbehbr_adbehkr_mcafeeus_mfeap_mfeeu.csv'


retry_success = pd.concat([retry_success_1, pd.read_csv(WORK_DIR + file_2)], axis=0)
retry_success.shape

/drhdev/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/drhdev/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (12,23,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(997957, 57)

In [59]:
retry_success =  pd.read_csv('/drhdev/workspace/ml/pdf_all_site_2018-05-01_2018-05-31_pro_att.csv')
# retry_success =  pd.read_csv('/drhdev/workspace/ml/pdf_all_site_2018-01-01_2018-03-31_pro_att.csv')
retry_success.shape

(1262284, 58)

In [4]:
import glob

path =r'/var/spark/ml_files' # use your path
allFiles = glob.glob(path + "/retry_success_2018*.csv")
retry_success = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
retry_success = pd.concat(list_)

retry_success

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (37,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Remove the CWD from sys.path while we load stuff.


,Unnamed: 0,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,days_between,event_uuid,exp_setting_id,failed_attempt_date,failed_day_of_month,failed_decline_type,failed_event_uuid,failed_response_code,failed_response_message,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,received_date,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,retry_rule_id,site_id,status,subscription_id,subsegment_id,success,time_of_day_rule_id,transaction_date,transaction_date_in_string,transaction_hour,transaction_id,transaction_type
0,0,NaN,NON3DS,TINKOFF BANK,RU,553691.0,MasterCard,MasterCard New World Immediate Debit,Consumer,NON USA Consumer Debit - No Pin Access,0822,NaN,NaN,4,thursday,7,9dafe40b-80c4-44c0-a7b8-bae9b8ff8b2b,NaN,2018-04-05 01:12:00,4,Insufficient Funds,dfb3010f-2876-44c2-a52d-ea70cc6f585f,27001,Transaction refused[51] [000000] [] [],Debit,RU,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,1519.84,23.524918,RUB,MasterCard,netgiro-seb,2018-04-12,2,1519.84,23.524918,RUB,46772409300,0,SUCCESS[00] [091571] [] [],NaN,adbehru,New,10410787601,15557791100,1,NaN,2018-04-12 01:22:06,2018-04-12 01:22:06,1,6659804773,Authorize
1,1,NaN,NON3DS,Caixa Economica Federal,BR,421965.0,Visa,Visa Gold,Consumer,True credit (No PIN/Signature capability),0322,NaN,NaN,4,saturday,5,16aab649-c437-46c0-9878-be2eb3881a4c,NaN,2018-04-23 01:05:20,4,do_not_honor,f18c8076-3054-44ed-a135-631f3b8fe620,530,7: Do Not Honor,Credit,BR,Avast Brazil,1883448942,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,79.00,22.707420,BRL,Visa,netgiro-br,2018-04-28,2,79.00,22.707420,BRL,47351118300,27001,Transaction refused[05] [null] [] [],NaN,avastbr,New,10047576801,13343435300,0,NaN,2018-04-28 01:28:38,2018-04-28 01:28:38,1,6717665123,Authorize
2,2,NaN,NON3DS,"Woori Card Co., Ltd.",KR,421420.0,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0122,NaN,NaN,4,monday,9,5d590cca-5f09-4f93-a20e-fa4934a64592,NaN,2018-04-14 01:18:27,4,Insufficient Funds,501a40b0-10e6-45bc-ab0a-33794732c398,27001,Transaction refused[51] [000000] [] [],Debit,KR,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Declined,DR_KOREA-ENTITY,23100.00,21.601956,KRW,Visa,netgiro-seb,2018-04-23,6,23100.00,21.601956,KRW,47132721400,27001,Transaction refused[51] [000000] [] [],NaN,adbehkr,New,9420922201,15464342300,0,NaN,2018-04-23 00:48:46,2018-04-23 00:48:46,0,6699112143,Authorize
3,3,NaN,NON3DS,AXIS BANK LIMITED,IN,529616.0,MasterCard,Premium Debit MasterCard Card,Consumer,NON USA Consumer Debit - No Pin Access,0620,NaN,NaN,4,sunday,7,edba7351-80ce-4fc5-ba88-d6eafcda3a17,NaN,2018-04-22 01:40:29,4,do_not_try,9209c095-a24a-44be-9655-92424160c0cf,27050,Do not try again/use alternate payment card.[57] [] [111800] [111800],Debit,IN,DRI*Adobe Sales,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,1596.54,23.948745,INR,MasterCard,netgiro-seb,2018-04-29,5,1596.54,23.948745,INR,47398402800,27001,Transaction refused[57] [000000] [] [],NaN,adbehap,New,11322986801,15520728400,0,NaN,2018-04-29 01:10:46,2018-04-29 01:10:46,1,6721850793,Authorize
4,4,NaN,NON3DS,"KB Kookmin Card Co., Ltd.",KR,457973.0,Visa,Visa Gold,Consumer,True credit (No PIN/Signature capability),0322,NaN,NaN,4,monday,7,07aed8ad-e8db-4d94-bbbe-92e2c90321ec,NaN,2018-04-09 01:28:24,4,txn_refused,4aee17f1-807d-4653-87f5-283c715a14a3,27001,Transaction refused[05] [000000] [] [],Credit,KR,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_KOREA-ENTITY,23100.00,21.615995,KRW,Visa,netgiro-seb,2018-04-16,4,23100.00,21.615995,KRW,46906367600,0,SUCCESS[00] [333099] [] [],NaN,adbehkr,New,10989500801,15486876600,1,NaN,2018-04-16 01:30:33,2018-04-16 01:30:33,1,6674669783,Authorize
5,5,NaN,CD606B6191

In [5]:
retry_success['day_of_week'].value_counts()

def is_weekend(day):
    """Determines whether the given day is weekend
    Args:
        day (str) : should be a lower case day_of_month. e.g: sunday, saturday 
    Return True (bool) if the day is either sunday or saturday
    """
    if day in ['saturday', 'sunday']:
        return True
    return False

is_weekend('sunday')

True

In [6]:
retry_success['is_weekend'] = retry_success['day_of_week'].apply(is_weekend)

In [54]:
retry_success[retry_success['received_date'].astype(str).str.startswith('201801')]

,Unnamed: 0,received_date,site_id,subscription_id,event_uuid,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,retry_rule_id,status,subsegment_id,time_of_day_rule_id,transaction_date,transaction_date_in_string,transaction_hour,transaction_id,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid,is_weekend,Mean,Min,Max,StdDev
4203270,0,20180120,adbehap,10004740401,285770ae-cc5a-41fe-8f91-37a3d0a37989,NaN,NON3DS,KRUNGTHAI CARD PUBLIC COMPANY LIMITED,TH,540716.0,MasterCard,Titanium MasterCard Card,Consumer,True credit (No PIN/Signature capability),0718,NaN,NaN,20,saturday,NaN,Credit,TH,DRI*Adobe Sales,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,284.80,8.933487,THB,MasterCard,netgiro-seb,2,284.80,8.933487,THB,44465456900,0,SUCCESS[00] [445629] [] [],NaN,New,14863200900,NaN,2018-01-20 07:43:01,2018-01-20 01:43:01,1,6388760073,Authorize,1,2018-01-15 01:16:13,15,27001,Transaction refused[05] [] [111800] [111800],txn_refused,5,2206daa6-c2c7-4f2f-b6e1-aef9b50cc67b,True,19.968746,8.916707,401.757536,22.661077
4203271,1,20180120,adbehap,10004740401,285770ae-cc5a-41fe-8f91-37a3d0a37989,NaN,NON3DS,KRUNGTHAI CARD PUBLIC COMPANY LIMITED,TH,540716.0,MasterCard,Titanium MasterCard Card,Consumer,True credit (No PIN/Signature capability),0718,NaN,NaN,20,saturday,NaN,Credit,TH,DRI*Adobe Sales,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,284.80,8.933487,THB,MasterCard,netgiro-seb,2,284.80,8.933487,THB,44465456900,0,SUCCESS[00] [445629] [] [],NaN,New,14863200900,NaN,2018-01-20 07:43:01,2018-01-20 01:43:01,1,6388760073,Authorize,1,2018-01-15 01:16:11,15,27001,Transaction refused[05] [000000] [] [],txn_refused,5,5ec1d160-6493-4819-a359-879d9104c091,True,19.968746,8.916707,401.757536,22.661077
4203272,2,20180120,adbehkr,10005736901,fc21ca90-86f1-4724-a044-043dc84d2771,NaN,NON3DS,"SHINHAN CARD CO., LTD.",KR,510737.0,MasterCard,Standard Debit MasterCard,Consumer,NON USA Consumer Debit - No Pin Access,0522,NaN,NaN,20,saturday,NaN,Debit,KR,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_KOREA-ENTITY,23100.00,21.660890,KRW,MasterCard,netgiro-seb,2,23100.00,21.660890,KRW,44465511000,0,SUCCESS[00] [672595] [] [],NaN,New,14863172700,NaN,2018-01-20 07:43:33,2018-01-20 01:43:33,1,6388764013,Authorize,1,2018-01-15 01:15:54,15,27054,Attempt lower amount.[51] [] [122800] [122800],Insufficient Funds,5,535fd627-5f0b-4233-a70c-977f1945e089,True,22.680035,9.900000,1519.492270,34.741722
4203273,3,20180120,adbehkr,10005736901,fc21ca90-86f1-4724-a044-043dc84d2771,NaN,NON3DS,"SHINHAN CARD CO., LTD.",KR,510737.0,MasterCard,Standard Debit MasterCard,Consumer,NON USA Consumer Debit - No Pin Access,0522,NaN,NaN,20,saturday,NaN,Debit,KR,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_KOREA-ENTITY,23100.00,21.660890,KRW,MasterCard,netgiro-seb,2,23100.00,21.660890,KRW,44465511000,0,SUCCESS[00] [672595] [] [],NaN,New,14863172700,NaN,2018-01-20 07:43:33,2018-01-20 01:43:33,1,6388764013,Authorize,1,2018-01-15 01:15:51,15,27001,Transaction refused[51] [000000] [] [],Insufficient Funds,5,c4f665a0-9119-44aa-8ba0-844a69a6da0f,True,22.680035,9.900000,1519.492270,34.741722
4203274,4,20180131,adbehkr,10019069001,52f12b93-33dd-429b-bd0b-298fffd97530,NaN,NON3DS,"SHINHAN CARD CO., LTD.",KR,510737.0,MasterCard,Standard Debit MasterCard,Consumer,NON USA Consumer Debit - No Pin Access,0818,NaN,NaN,31,wednesday,NaN,Debit,KR,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_KOREA-ENTITY,35200.00,32

In [7]:
grouped = retry_success['payment_amount_usd'].groupby(retry_success['bin']).agg([('Mean','mean'),('Min','min'),('Max','max'),('StdDev','std'),'count']).reset_index()
bin_profile = grouped[grouped['count'] >= 20]
bin_profile

,bin,Mean,Min,Max,StdDev,count
27,354035.0,119.767392,18.620400,572.349627,168.541284,22
43,354101.0,386.251321,18.716432,1827.651383,638.308094,20
44,354102.0,280.626165,19.073195,1824.112097,354.089905,30
67,358277.0,55.601248,18.241768,424.956307,100.615073,47
69,358403.0,58.505398,17.981433,428.518055,92.698876,78
90,358787.0,175.512610,18.096556,432.736364,191.300609,25
92,360218.0,46.278951,9.990000,189.990000,33.997189,162
96,360755.0,31.197610,15.545479,73.029615,11.144235,32
100,360857.0,46.423983,9.990000,359.880000,58.768888,118
126,364172.0,38.436897,9.990000,79.990000,20.575167,29


In [8]:
retry_success = pd.merge(retry_success, bin_profile[['bin','Mean','Min','Max','StdDev']], left_on='bin', right_on='bin', how='left')
retry_success.shape

(5976040, 61)

In [9]:
### resampling, balancing classes ###
def balancing_class(df_ml):
    print(df_ml[LABEL].value_counts(normalize=True))
    df_cls_0 = df_ml[df_ml[LABEL] == 0]
    df_cls_1 = df_ml[df_ml[LABEL] == 1]

    #over sampling
# #     df_cls_0_over = df_cls_0.sample(len(df_cls_1), replace=True)
# #     df_ml_bl = pd.concat([df_cls_0_over, df_cls_1], axis=0)
    df_cls_1_over = df_cls_1.sample(len(df_cls_0), replace=True)
    df_ml_bl = pd.concat([df_cls_1_over, df_cls_0], axis=0)

#     #under sampling
#     df_cls_1_under = df_cls_1.sample(len(df_cls_0), replace=True)
#     df_ml_bl = pd.concat([df_cls_1_under, df_cls_0], axis=0)
#     df_cls_0_under = df_cls_0.sample(len(df_cls_1), replace=True)
#     df_ml_bl = pd.concat([df_cls_0_under, df_cls_1], axis=0)
    
    print(df_ml_bl[LABEL].value_counts(normalize=True))
    print(df_ml_bl.shape)
    return df_ml_bl

In [11]:
target_size=1500000

### resampling, balancing classes ###
def balancing_class(df_ml):
    print(df_ml[LABEL].value_counts(normalize=True))
    df_cls_0 = df_ml[df_ml[LABEL] == 0]
    df_cls_1 = df_ml[df_ml[LABEL] == 1]

    #over sampling
# #     df_cls_0_over = df_cls_0.sample(len(df_cls_1), replace=True)
# #     df_ml_bl = pd.concat([df_cls_0_over, df_cls_1], axis=0)
    df_cls_1_over = df_cls_1.sample(target_size, replace=True)
    df_cls_0_under = df_cls_0.sample(target_size, replace=True)
    df_ml_bl = pd.concat([df_cls_1_over, df_cls_0_under], axis=0)

#     #under sampling
# #     df_cls_1_under = df_cls_1.sample(len(df_cls_0), replace=True)
# #     df_ml_bl = pd.concat([df_cls_1_under, df_cls_0], axis=0)
#     df_cls_0_under = df_cls_0.sample(len(df_cls_1), replace=True)
#     df_ml_bl = pd.concat([df_cls_0_under, df_cls_1], axis=0)
    
    print(df_ml_bl[LABEL].value_counts(normalize=True))
    print(df_ml_bl.shape)
    return df_ml_bl

In [ ]:
retry_success_balanced_all = balancing_class(retry_success)
# csv_name = str('subs_retry_balanced__%s_%s_%s.csv' % (start_date, end_date, "_".join(site_ids)))
# retry_success_balanced_all.to_csv(WORK_DIR + csv_name)
# print("Saved into " + WORK_DIR + csv_name)
# retry_success_balanced_all.shape

0    0.883412
1    0.116588
Name: success, dtype: float64


In [13]:
retry_success_balanced_all = retry_success_balanced_all.drop('Unnamed: 0', axis=1)
retry_success_balanced_all.head()

,received_date,site_id,subscription_id,event_uuid,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,retry_rule_id,status,subsegment_id,time_of_day_rule_id,transaction_date,transaction_date_in_string,transaction_hour,transaction_id,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid,is_weekend,Mean,Min,Max,StdDev
541107,20180512,adbehkr,10035898601,79cd0613-53e1-4571-ab74-c654fb6e25ad,NaN,NON3DS,"KB KOOKMIN CARD CO., LTD",KR,527289.0,MasterCard,Standard Debit MasterCard,Consumer,NON USA Consumer Debit - No Pin Access,0622,NaN,NaN,12,saturday,NaN,Debit,KR,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_KOREA-ENTITY,11000.00,10.311609,KRW,MasterCard,netgiro-seb,3,11000.00,10.311609,KRW,47840486700,0,SUCCESS[00] [321439] [] [],NaN,New,15773250800,NaN,2018-05-12 06:01:50,2018-05-12 01:01:50,1,6770275373,Authorize,1,2018-05-05 01:07:51,5,27054,Attempt lower amount.[61] [] [122800] [122800],attempt_lower_amount,7,6734e7c1-a106-404b-b142-342e52ffb48c,True,21.445500,9.089865,396.906303,21.483283
4407421,20180117,adbehil,9684697401,82b25c0a-6de0-4319-ae42-37a5ff670cb7,NaN,NON3DS,EUROPAY (EUROCARD) ISRAEL LTD.,IL,532610.0,MasterCard,Gold MasterCard Card,Consumer,True credit (No PIN/Signature capability),1219,NaN,NaN,17,wednesday,NaN,Credit,IL,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,175.00,51.243835,ILS,MasterCard,netgiro-seb,2,175.00,51.243835,ILS,44388457000,0,SUCCESS[00] [963977] [] [],NaN,New,14841043300,NaN,2018-01-17 07:14:06,2018-01-16 01:13:38,1,6380071353,Authorize,1,2018-01-12 01:35:38,12,27054,Attempt lower amount.[51] [] [111800] [111800],Insufficient Funds,4,c94ddb82-c5bb-4fb3-af73-be6dadc294e1,False,32.905834,2.490000,612.479142,49.427762
4952948,20180108,kasperus,3854295901,ab6025c7-966c-4401-bc93-d12e61061d8a,NaN,CDC4BC59F3197CCA,NaN,US,371720.0,American Express,NaN,NaN,True credit (No PIN/Signature capability),1022,NaN,NaN,8,monday,NaN,NaN,NaN,DRI*Kaspersky Lab,941000108071_00000001,DR_INC-ENTITY,Completed,DR_INC-ENTITY,59.99,59.990000,USD,AmericanExpress,mes,4,59.99,59.990000,USD,44148013600,100,SUCCESS Exact Match,NaN,New,6133656400,NaN,2018-01-08 06:38:19,2018-01-08 00:38:19,0,6353068573,Authorize,1,2018-01-01 00:49:16,1,530,005: Do not honor,do_not_honor,7,c0c03742-66c0-41eb-a9db-8adaf9ec408f,False,85.445135,10.870000,162.360000,29.261621
1717052,20180410,adbehru,10403382601,b4c08f4d-6411-40f4-b451-3ea1bfa8f951,NaN,NON3DS,SBERBANK OF RUSSIA,RU,546940.0,MasterCard,Platinum MasterCard Salary Immediate Debit,Consumer,NON USA Consumer Debit - No Pin Access,1119,NaN,NaN,10,tuesday,NaN,Debit,RU,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,759.92,12.615945,RUB,MasterCard,netgiro-seb,2,759.92,12.615945,RUB,46708707600,0,SUCCESS[00] [213085] [] [],NaN,New,15538264300,NaN,2018-04-10 06:13:57,2018-04-10 01:13:57,1,6652776663,Authorize,1,2018-04-03 01:18:17,3,27001,Transaction refused[51] [000000] [] [],Insufficient Funds,7,9e01f8e3-55f5-46f3-89c3-27ce5226fee7,False,25.629998,1.847553,805.678577,42.429407
3697419,20180219,avast,3989976101,9b3b8779-879b-4998-9643-a49bda137d5a,NaN,NON3DS,Santander UK Plc,GB,475714.0,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0720,RETRY_DECLINED.2,NaN,19,monday,NaN,Debit,GB,DRI*AVAST Software,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,39.99,56.128212,GBP,Visa,netgiro-seb,2,39.99,56.128212,GBP,45259737400,0,SUCCESS[00] [440221] [] [],NaN,New,1287

In [15]:
retry_success_balanced_all.to_csv(WORK_DIR + 'best_retry_with_is_weekend.csv')

In [14]:
original_size = len(retry_success)
balanced_size = len(retry_success_balanced_all)
fail_size = retry_success[LABEL].value_counts(normalize=True)[0.0]
success_size =  retry_success[LABEL].value_counts(normalize=True)[1.0]

In [15]:
#import for training
import numpy as np
from sklearn import cross_validation
from sklearn import metrics
from sklearn.model_selection import cross_val_score

from sklearn import linear_model
from sklearn import tree
from sklearn import cross_validation
from sklearn import ensemble
from sklearn import linear_model
from sklearn import svm
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
# from spark_sklearn import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsClassifier

from utils import PreProcessing
from encoder import SafeLabelEncoder

/drhdev/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
from sklearn import cross_validation

def print_accuracy_report(classifier, X, y, num_validations=5):
    accuracy = cross_validation.cross_val_score(classifier, 
            X, y, scoring='accuracy', cv=num_validations)
    print("CV Accuracy: " + str(round(100*accuracy.mean(), 2)) + "%")

    f1 = cross_validation.cross_val_score(classifier, 
            X, y, scoring='f1_weighted', cv=num_validations)
    print("CV F1: " + str(round(100*f1.mean(), 2)) + "%")

    precision = cross_validation.cross_val_score(classifier, 
            X, y, scoring='precision_weighted', cv=num_validations)
    print("CV Precision: " + str(round(100*precision.mean(), 2)) + "%")

    recall = cross_validation.cross_val_score(classifier, 
            X, y, scoring='recall_weighted', cv=num_validations)
    print("CV Recall: " + str(round(100*recall.mean(), 2)) + "%")

In [80]:
# FEATURES_CAT = ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage',
#                  'issuer_country', 'first_response_code', 'day_of_month', 'site_id', 'billing_country',
#                'first_decline_type',  'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'first_day_of_month']

FEATURES_CAT = ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage',
                 'issuer_country', 'failed_response_code', 'day_of_month', 'site_id', 'billing_country',
               'failed_decline_type',  'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 
                'renew_att_num', 'failed_day_of_month', 'is_weekend']

FEATURES_NUM = ['payment_amount_usd']
FEATURES_ENCODED = [ 'week_of_month', 'day_of_week', 'Mean', 'Min', 'Max', 'StdDev']
FEATURES = FEATURES_CAT + FEATURES_ENCODED


from sklearn.model_selection import GridSearchCV
# from spark_sklearn import GridSearchCV

# from training import PreProcessing
from sklearn.pipeline import make_pipeline


FIELDS = FEATURES_CAT + FEATURES_NUM + ['transaction_date_in_string']

features_dict = {'FEATURES_CAT': FEATURES_CAT, 'FEATURES_NUM':FEATURES_NUM, 'FEATURES_ENCODED':FEATURES_ENCODED}
features_dict_key = 'preprocessing__features_dict'
def display_feature_importance(pipe, model_name, df_features):
    classifier = pipe.named_steps[model_name]
    feature_importance = classifier.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    print("feature_importance column ",df_features.columns[sorted_idx])
    print("feature_importance val ",feature_importance[sorted_idx])
    pos = np.arange(sorted_idx.shape[0]) + .5
    pvals = feature_importance[sorted_idx]
    pcols = df_features.columns[sorted_idx]
    plt.figure(figsize=(8,12))
    plt.barh(pos, pvals, align='center')
    plt.yticks(pos, pcols)
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')

def build_and_train(df, clf, param_grid, model_name, model_file = '', best_param=None):
    model_prefix = model_name + '__'
    time_start = time.time()
    df_X = df[FIELDS]
    result_dict = {}
    x_train, x_test, y_train, y_test = cross_validation.train_test_split(df_X, df[LABEL], \
                                                        test_size=0.25, random_state=42)

    
    if best_param is None:
    
        pipe = make_pipeline(PreProcessing(bin_profile), clf())
        score = 'accuracy' #  ['accuracy','precision_macro', 'recall_macro', 'f1_macro']
        print("# Tuning hyper-parameters for %s" % score)
        
        pipe_param_grid = {model_prefix + k: v for k, v in param_grid.items()}
        print("pipe_param_grid ", pipe_param_grid)
#         clf_gs = GridSearchCV(sc, pipe, pipe_param_grid, cv=3, scoring=score, n_jobs=-1, fit_params={features_dict_key: features_dict})
        clf_gs = GridSearchCV(pipe, pipe_param_grid, cv=3, scoring=score, n_jobs=-1, fit_params={features_dict_key: features_dict})
        clf_gs.fit(x_train, y_train)

        print("# Best parameters set found on development set:")
        print(clf_gs.best_params_)
        result_dict['hyper_params'] = clf_gs.best_params_
        print("# Grid scores on development set:")
        means = clf_gs.cv_results_['mean_test_score']
        stds = clf_gs.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf_gs.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

        best_parameters = clf_gs.best_params_
        best_parameters = {k.replace(model_prefix,''): v for k, v in best_parameters.items()}
        
    else:
        best_parameters= best_param
   
    print("best_parameters ", best_parameters)        
    print("x_train", x_train.shape)
    print("x_test", x_test.shape)

    if model_file != '':
#     if model_name == 'xgbclassifier':
#         best_parameters['process_type'] = 'update'
        best_parameters['updater'] = 'refresh'
        best_parameters['refresh_leaf'] = True
        

    pipe = make_pipeline(PreProcessing(bin_profile), clf(**best_parameters))
    
    if model_name == 'xgbclassifier':
        print("training xgb ....... ")
        if model_file == '':
            pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict, xgbclassifier__eval_metric='error')
        else:
            print("Using model_file to train: ", model_file)
            pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict, xgbclassifier__xgb_model=model_file, xgbclassifier__eval_metric='error')
    else:        
        pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict)
        
    preprocess = pipe.named_steps['preprocessing']

        
    y_pred_train = pipe.predict(x_train).round()
    y_pred_test = pipe.predict(x_test).round()

    training_time = time.time() - time_start
    print("# training time:", training_time)
    result_dict['training_time'] = training_time
    clf_d = DummyClassifier(strategy='most_frequent')
    dummy_pipe = make_pipeline(PreProcessing(bin_profile), clf_d)
    dummy_pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict)
    y_pred_test_dummy = dummy_pipe.predict(x_test).round()
    
    accuracy_dummy = metrics.accuracy_score(y_test, y_pred_test_dummy)
    train_accuracy = metrics.accuracy_score(y_train, y_pred_train)
    test_accuracy = metrics.accuracy_score(y_test, y_pred_test)
    
    
    
    
    result_dict['accuracy_dummy'] = accuracy_dummy
    result_dict['train_accuracy'] = train_accuracy
    result_dict['test_accuracy'] = test_accuracy
#     print_accuracy_report(pipe, x_train, y_train, num_validations=3)
#     print(metrics.classification_report(y_test, y_pred_test))
    train_class_report = metrics.classification_report(y_train, y_pred_train)
    test_class_report = metrics.classification_report(y_test, y_pred_test)
    result_dict['train_class_report'] = train_class_report
    result_dict['test_class_report'] = test_class_report
    conf_mx = metrics.confusion_matrix(y_test, y_pred_test)
    
    print("accuracy_dummy:", accuracy_dummy)
    print("training accuracy:", train_accuracy)
    print(train_class_report)
    print("test accuracy:", test_accuracy)
    print(test_class_report)
    print("# confusion_matrix -  test:\n", conf_mx)
    result_dict['conf_mx'] = conf_mx

    return pipe, result_dict


In [59]:
# save model
from utils import PreProcessing
from sklearn.externals import joblib
import os
from os import path

MODEL_DIR = WORK_DIR + "models"


def write_model(model, model_name, idx=None): 
    build_id = "" if idx is None else "_" + str(idx)
    file_name = path.join(MODEL_DIR, '%s%s.pkl' % (model_name, build_id))
    if not os.path.exists(os.path.dirname(file_name)):
        os.makedirs(os.path.dirname(file_name))
    file = joblib.dump(model, file_name)
    return (file, file_name)

In [19]:
import datetime
from cassandra.cluster import Cluster
cassandra_endpoint = '10.224.12.32'
cluster = Cluster([cassandra_endpoint])
mlp_session = cluster.connect('mlpks')


def insert_model_info(model_id, version, file_name, desc, model_type=MODEL_TYPE, algorithm='XGBClassifier', hyper_parameter=None, eval_metrics=None):
#     file_name = model_id + '.' + str(version) + ".pkl"
    
    mlp_session.execute(
    """
    INSERT INTO ml_model_storage (model_type, model_id, version, features_cat, features_encoded, features_num, repo_path, description, creation_date, modification_date, algorithm, hyper_parameter, eval_metrics)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """,
    (model_type, model_id, version, str(FEATURES_CAT), str(FEATURES_ENCODED), str(FEATURES_NUM), file_name, desc, datetime.datetime.utcnow(), datetime.datetime.utcnow(), algorithm, hyper_parameter, eval_metrics)
        
    )
    print("Model %s version %d is inserted into model repo" % (model_id, version))      


In [20]:
def get_latest_version(model_id, model_type=MODEL_TYPE):
    """Get latest version of the given model_id"""
    latest_version_query = "select version from ml_model_storage  where model_type = '%s' and model_id = '%s' limit 1" % (model_type, model_id)
    query_result = mlp_session.execute(latest_version_query).one()
    if query_result is None:
        latest_version = 1
    else:
        latest_version = query_result.version
    
    return latest_version


In [21]:
REPOSITORY_URL = 'http://nexus.digitalriverws.net/nexus'
REPO_USER = 'deployment'
REPO_PWD = 'deployment123'
REPO_ID = 'foundationreleases'
REPO_GROUP = 'com.digitalriver.prediction-service'

import repositorytools

def upload_artifact(file_path):
    """Upload artifact to Nexus Repo"""
    artifact = repositorytools.LocalArtifact(local_path=file_path, group=REPO_GROUP)

    client = repositorytools.repository_client_factory(repository_url=REPOSITORY_URL, user=REPO_USER, password=REPO_PWD)
    remote_artifacts = client.upload_artifacts(local_artifacts=[artifact], repo_id=REPO_ID, use_direct_put=True)
    print(remote_artifacts)
    return str(remote_artifacts[0]) if remote_artifacts else ''

In [43]:
""" Train the model"""

#XGBoost Classifier
from xgboost import XGBClassifier
# import xgboost as xgb
classifier = XGBClassifier

tuned_parameters = {
              'objective':['binary:logistic'],
              'learning_rate': [0.2], #so called `eta` value
              'max_depth': [10],
              'min_child_weight': [11],
              'silent': [0],
              'subsample': [0.5],
              'colsample_bytree': [0.7],
#               'n_estimators': [500, 1000], #number of trees, change it to 1000 for better results
              'n_estimators': [1000], #number of trees, change it to 1000 for better results  
              'missing':[-999],
              'seed': [1337]}

best_parameters = {
              'objective': 'binary:logistic',
              'learning_rate': 0.2, #so called `eta` value
              'max_depth': 10,
              'min_child_weight': 11,
              'silent': 0,
              'subsample': 0.5,
              'colsample_bytree': 0.7,
#               'n_estimators': [500, 1000], #number of trees, change it to 1000 for better results
              'n_estimators': 1000, #number of trees, change it to 1000 for better results  
              'missing':-999,
              'seed': 1337}

model_file = ''
# for idx, df_2018_f in enumerate(df_2018_filtered):
#     print("Iteration: ", idx)
model_id = 'ML-BR-1'
version = get_latest_version(model_id) + 1
model_name = model_id + '.' + str(version)
xgb_clf, result_d = build_and_train(retry_success_balanced_all[:100], classifier, tuned_parameters, 'xgbclassifier', model_file, best_param=best_parameters)
model_file, model_file_name = write_model(xgb_clf, model_name)
print("model_file is generated: ", model_file)
print("result_dict: ", result_d)


best_parameters  {'objective': 'binary:logistic', 'learning_rate': 0.2, 'max_depth': 10, 'min_child_weight': 11, 'silent': 0, 'subsample': 0.5, 'colsample_bytree': 0.7, 'n_estimators': 1000, 'missing': -999, 'seed': 1337}
x_train (75, 19)
x_test (25, 19)
training xgb ....... 
features_dict:  {'FEATURES_CAT': ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'failed_response_code', 'day_of_month', 'site_id', 'billing_country', 'failed_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month'], 'FEATURES_NUM': ['payment_amount_usd'], 'FEATURES_ENCODED': ['week_of_month', 'day_of_week', 'Mean', 'Min', 'Max', 'StdDev', 'is_weekend']}
---------     card_brand funding_source                          card_category  \
0        Visa          Debit                           Visa Classic   
1        Visa          Debit                           Visa Classic   
2  MasterCard         Credi

KeyError: 'is_weekend'

In [60]:

""" Train the model"""

#XGBoost Classifier
from xgboost import XGBClassifier
# import xgboost as xgb
classifier = XGBClassifier

tuned_parameters = {
              'objective':['binary:logistic'],
              'learning_rate': [0.2], #so called `eta` value
              'max_depth': [10],
              'min_child_weight': [11],
              'silent': [0],
              'subsample': [0.5],
              'colsample_bytree': [0.7],
#               'n_estimators': [500, 1000], #number of trees, change it to 1000 for better results
              'n_estimators': [1000], #number of trees, change it to 1000 for better results  
              'missing':[-999],
              'seed': [1337]}

best_parameters = {
              'objective': 'binary:logistic',
              'learning_rate': 0.2, #so called `eta` value
              'max_depth': 10,
              'min_child_weight': 11,
              'silent': 0,
              'subsample': 0.5,
              'colsample_bytree': 0.7,
#               'n_estimators': [500, 1000], #number of trees, change it to 1000 for better results
              'n_estimators': 1000, #number of trees, change it to 1000 for better results  
              'missing':-999,
              'seed': 1337}

model_file = ''
model_id = 'ML-BR-1'
version = get_latest_version(model_id) + 1

# for idx, df_2018_f in enumerate(df_2018_filtered):
#     print("Iteration: ", idx)
dates = ['201801','201802','201803','201804','201805','201806']
for idx, date in enumerate(dates):
    print(date)
    model_name = model_id + '.' + str(version) + '.' + str(idx)
    print(model_name)
    retry_success_ = retry_success[retry_success['received_date'].astype(str).str.startswith(date)]
    retry_success_balanced_all_ = balancing_class(retry_success_)
    xgb_clf, result_d = build_and_train(retry_success_balanced_all_, classifier, tuned_parameters, 'xgbclassifier', model_file, best_param=best_parameters)
    model_file, model_file_name = write_model(xgb_clf, model_name)
    print("model_file is generated: ", model_file)
    print("result_dict: ", result_d)


201801
ML-BR-1.41.0
0    0.885533
1    0.114467
Name: success, dtype: float64
1    0.5
0    0.5
Name: success, dtype: float64
(185072, 61)
best_parameters  {'objective': 'binary:logistic', 'learning_rate': 0.2, 'max_depth': 10, 'min_child_weight': 11, 'silent': 0, 'subsample': 0.5, 'colsample_bytree': 0.7, 'n_estimators': 1000, 'missing': -999, 'seed': 1337}
x_train (138804, 20)
x_test (46268, 20)
training xgb ....... 
features_dict:  {'FEATURES_CAT': ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'failed_response_code', 'day_of_month', 'site_id', 'billing_country', 'failed_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month', 'is_weekend'], 'FEATURES_NUM': ['payment_amount_usd'], 'FEATURES_ENCODED': ['week_of_month', 'day_of_week', 'Mean', 'Min', 'Max', 'StdDev']}
self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issue

self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'failed_response_code', 'day_of_month', 'site_id', 'billing_country', 'failed_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month', 'is_weekend', 'week_of_month', 'day_of_week', 'Mean', 'Min', 'Max', 'StdDev', 'payment_amount_usd']
# training time: 868.5848796367645
features_dict:  {'FEATURES_CAT': ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'failed_response_code', 'day_of_month', 'site_id', 'billing_country', 'failed_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month', 'is_weekend'], 'FEATURES_NUM': ['payment_amount_usd'], 'FEATURES_ENCODED': ['week_of_month', 'day_of_week', 'Mean', 'Min', 'Max', 'StdDev']}
self.features_list:  ['card_brand', 'funding_source', 'card_category', 'ca

self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'failed_response_code', 'day_of_month', 'site_id', 'billing_country', 'failed_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month', 'is_weekend', 'week_of_month', 'day_of_week', 'Mean', 'Min', 'Max', 'StdDev', 'payment_amount_usd']
accuracy_dummy: 0.49990128888512847
training accuracy: 0.9891417773641312
             precision    recall  f1-score   support

          0       0.99      0.98      0.99    106378
          1       0.98      0.99      0.99    106364

avg / total       0.99      0.99      0.99    212742

test accuracy: 0.9349493752996587
             precision    recall  f1-score   support

          0       0.95      0.92      0.93     35450
          1       0.92      0.95      0.94     35464

avg / total       0.94      0.93      0.93     70914

# confusion_matrix -  test:
 [[32665  2785]

In [64]:

""" Train the model"""

#XGBoost Classifier
from xgboost import XGBClassifier
# import xgboost as xgb
classifier = XGBClassifier

tuned_parameters = {
              'objective':['binary:logistic'],
              'learning_rate': [0.2], #so called `eta` value
              'max_depth': [10],
              'min_child_weight': [11],
              'silent': [0],
              'subsample': [0.5],
              'colsample_bytree': [0.7],
#               'n_estimators': [500, 1000], #number of trees, change it to 1000 for better results
              'n_estimators': [1000], #number of trees, change it to 1000 for better results  
              'missing':[-999],
              'seed': [1337]}

best_parameters = {
              'objective': 'binary:logistic',
              'learning_rate': 0.2, #so called `eta` value
              'max_depth': 10,
              'min_child_weight': 11,
              'silent': 0,
              'subsample': 0.5,
              'colsample_bytree': 0.7,
#               'n_estimators': [500, 1000], #number of trees, change it to 1000 for better results
              'n_estimators': 1000, #number of trees, change it to 1000 for better results  
              'missing':-999,
              'seed': 1337}

model_file = ''
model_id = 'ML-BR-1'
version = get_latest_version(model_id) + 1

# for idx, df_2018_f in enumerate(df_2018_filtered):
#     print("Iteration: ", idx)
dates = ['201801','201802','201803','201804','201805','201806']
for idx, date in enumerate(dates):
    print(date)
    model_name = model_id + '.' + str(version) + '.' + str(idx)
    print(model_name)
    retry_success_ = retry_success[retry_success['received_date'].astype(str).str.startswith(date)]
    retry_success_balanced_all_ = balancing_class(retry_success_)
    xgb_clf, result_d = build_and_train(retry_success_balanced_all_, classifier, tuned_parameters, 'xgbclassifier', model_file, best_param=best_parameters)
    model_file, model_file_name = write_model(xgb_clf, model_name)
    print("model_file is generated: ", model_file)
    print("result_dict: ", result_d)


201801
ML-BR-1.42.0
0    0.885533
1    0.114467
Name: success, dtype: float64
1    0.5
0    0.5
Name: success, dtype: float64
(1431738, 61)
best_parameters  {'objective': 'binary:logistic', 'learning_rate': 0.2, 'max_depth': 10, 'min_child_weight': 11, 'silent': 0, 'subsample': 0.5, 'colsample_bytree': 0.7, 'n_estimators': 1000, 'missing': -999, 'seed': 1337}
x_train (1073803, 20)
x_test (357935, 20)
training xgb ....... 
features_dict:  {'FEATURES_CAT': ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'failed_response_code', 'day_of_month', 'site_id', 'billing_country', 'failed_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month', 'is_weekend'], 'FEATURES_NUM': ['payment_amount_usd'], 'FEATURES_ENCODED': ['week_of_month', 'day_of_week', 'Mean', 'Min', 'Max', 'StdDev']}
self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'is

self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'failed_response_code', 'day_of_month', 'site_id', 'billing_country', 'failed_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month', 'is_weekend', 'week_of_month', 'day_of_week', 'Mean', 'Min', 'Max', 'StdDev', 'payment_amount_usd']
# training time: 7632.863489866257
features_dict:  {'FEATURES_CAT': ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'failed_response_code', 'day_of_month', 'site_id', 'billing_country', 'failed_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month', 'is_weekend'], 'FEATURES_NUM': ['payment_amount_usd'], 'FEATURES_ENCODED': ['week_of_month', 'day_of_week', 'Mean', 'Min', 'Max', 'StdDev']}
self.features_list:  ['card_brand', 'funding_source', 'card_category', 'ca

self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'failed_response_code', 'day_of_month', 'site_id', 'billing_country', 'failed_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month', 'is_weekend', 'week_of_month', 'day_of_week', 'Mean', 'Min', 'Max', 'StdDev', 'payment_amount_usd']
accuracy_dummy: 0.4997393918190451
training accuracy: 0.9897291816903118
             precision    recall  f1-score   support

          0       1.00      0.98      0.99    840488
          1       0.98      1.00      0.99    840196

avg / total       0.99      0.99      0.99   1680684

test accuracy: 0.9834460255467417
             precision    recall  f1-score   support

          0       1.00      0.97      0.98    279968
          1       0.97      1.00      0.98    280260

avg / total       0.98      0.98      0.98    560228

# confusion_matrix -  test:
 [[271425   8543

In [ ]:

""" Train the model"""

#XGBoost Classifier
from xgboost import XGBClassifier
# import xgboost as xgb
classifier = XGBClassifier

tuned_parameters = {
              'objective':['binary:logistic'],
              'learning_rate': [0.2], #so called `eta` value
              'max_depth': [10],
              'min_child_weight': [11],
              'silent': [0],
              'subsample': [0.5],
              'colsample_bytree': [0.7],
#               'n_estimators': [500, 1000], #number of trees, change it to 1000 for better results
              'n_estimators': [1000], #number of trees, change it to 1000 for better results  
              'missing':[-999],
              'seed': [1337],
              'process_type': ['update'],
              'updater'     : ['refresh'],
               'refresh_leaf': [True]}
 

best_parameters = {
              'objective': 'binary:logistic',
              'learning_rate': 0.2, #so called `eta` value
              'max_depth': 10,
              'min_child_weight': 11,
              'silent': 0,
              'subsample': 0.5,
              'colsample_bytree': 0.7,
#               'n_estimators': [500, 1000], #number of trees, change it to 1000 for better results
              'n_estimators': 1000, #number of trees, change it to 1000 for better results  
              'missing':-999,
              'seed': 1337
               }


model_file = ''
model_id = 'ML-BR-1'
version = get_latest_version(model_id) + 1

# for idx, df_2018_f in enumerate(df_2018_filtered):
#     print("Iteration: ", idx)
dates = ['201801','201802','201803','201804','201805','201806']
for idx, date in enumerate(dates):
    print(date)
    model_name = model_id + '.' + str(version) + '.' + str(idx)
    print(model_name)
    retry_success_ = retry_success[retry_success['received_date'].astype(str).str.startswith(date)]
    retry_success_balanced_all_ = balancing_class(retry_success_)
    xgb_clf, result_d = build_and_train(retry_success_balanced_all_, classifier, tuned_parameters, 'xgbclassifier', model_file, best_param=best_parameters)
    model_file, model_file_name = write_model(xgb_clf, model_name)
    print("model_file is generated: ", model_file)
    print("result_dict: ", result_d)


201801
ML-BR-1.43.0
0    0.885533
1    0.114467
Name: success, dtype: float64
1    0.5
0    0.5
Name: success, dtype: float64
(1431738, 61)
best_parameters  {'objective': 'binary:logistic', 'learning_rate': 0.2, 'max_depth': 10, 'min_child_weight': 11, 'silent': 0, 'subsample': 0.5, 'colsample_bytree': 0.7, 'n_estimators': 1000, 'missing': -999, 'seed': 1337}
x_train (1073803, 20)
x_test (357935, 20)
training xgb ....... 
features_dict:  {'FEATURES_CAT': ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'failed_response_code', 'day_of_month', 'site_id', 'billing_country', 'failed_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month', 'is_weekend'], 'FEATURES_NUM': ['payment_amount_usd'], 'FEATURES_ENCODED': ['week_of_month', 'day_of_week', 'Mean', 'Min', 'Max', 'StdDev']}
self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'is

In [65]:
"""Upload model to Nexus repo and insert the model info into Cassandra table"""

repo_path = upload_artifact(model_file_name)
size_desc = str(", original size: %s (fail: %s, success: %s), balanced_size: %s" % (original_size, fail_size, success_size, balanced_size))
desc = desc + size_desc
hyper_params = result_d.pop('hyper_params', None)
insert_model_info(model_id, version, repo_path, desc=desc, eval_metrics=str(result_d), 
                  hyper_parameter=str(hyper_params))

http://nexus.digitalriverws.net/nexus/content/repositories/foundationreleases/com/digitalriver/prediction-service/ML-BR/1.42.5/ML-BR-1.42.5.pkl
[com.digitalriver.prediction-service:ML-BR:1.42.5::pkl]
Model ML-BR-1 version 42 is inserted into model repo


The following files were uploaded to repository foundationreleases
